In [1]:
import os.path
import random
import time
from functools import partial
from tkinter import Button, Tk, Entry


In [2]:

class EightPuzzle:
    def __init__(self, initial):
        self.initial = initial

    def actions(self, state):
        blank_index = state.index(0)
        row, col = divmod(blank_index, 3)
        possible_actions = []

        if row > 0:
            possible_actions.append('UP')
        if row < 2:
            possible_actions.append('DOWN')
        if col > 0:
            possible_actions.append('LEFT')
        if col < 2:
            possible_actions.append('RIGHT')

        return possible_actions

    def result(self, state, action):
        blank_index = state.index(0)
        row, col = divmod(blank_index, 3)

        if action == 'UP':
            new_blank_index = (row - 1) * 3 + col
        elif action == 'DOWN':
            new_blank_index = (row + 1) * 3 + col
        elif action == 'LEFT':
            new_blank_index = row * 3 + (col - 1)
        elif action == 'RIGHT':
            new_blank_index = row * 3 + (col + 1)

        new_state = list(state)
        new_state[blank_index], new_state[new_blank_index] = new_state[new_blank_index], new_state[blank_index]
        return tuple(new_state)

def manhattan_distance(state):
    total_distance = 0
    for num in range(1, 9):
        current_index = state.index(num)
        goal_index = (num - 1) if num != 0 else 8
        goal_row, goal_col = divmod(goal_index, 3)
        current_row, current_col = divmod(current_index, 3)
        total_distance += abs(goal_row - current_row) + abs(goal_col - current_col)
    return total_distance


In [3]:
def misplaced_tiles(state):
    return sum(1 if state[i] != i + 1 else 0 for i in range(8))

def astar_search(puzzle, heuristic):
    start_state = puzzle.initial
    goal_state = tuple(range(1, 9)) + (0,)
    explored = set()
    frontier = [(0 + heuristic(start_state), start_state, [])]

    while frontier:
        frontier.sort()
        cost, current_state, path = frontier.pop(0)

        if current_state == goal_state:
            return path

        if current_state not in explored:
            explored.add(current_state)

            for action in puzzle.actions(current_state):
                new_state = puzzle.result(current_state, action)
                new_cost = len(path) + 1 + heuristic(new_state)
                frontier.append((new_cost, new_state, path + [action]))

    return []

def greedy_best_first_search(puzzle, heuristic):
    start_state = puzzle.initial
    goal_state = tuple(range(1, 9)) + (0,)
    explored = set()
    frontier = [(heuristic(start_state), start_state, [])]

    while frontier:
        frontier.sort()
        current_heuristic, current_state, path = frontier.pop(0)

        if current_state == goal_state:
            return path

        if current_state not in explored:
            explored.add(current_state)

            for action in puzzle.actions(current_state):
                new_state = puzzle.result(current_state, action)
                new_heuristic = heuristic(new_state)
                frontier.append((new_heuristic, new_state, path + [action]))

    return []

def solve(start_state, goal_state):
    puzzle = EightPuzzle(tuple(start_state))
    p = puzzle.initial

    # Call A* search with Manhattan distance heuristic
    solution_astar_manhattan = astar_search(puzzle, manhattan_distance)

    # Call A* search with misplaced tiles heuristic
    solution_astar_misplaced = astar_search(puzzle, misplaced_tiles)

    # Call Greedy Best-First search with Manhattan distance heuristic
    solution_gbfs_manhattan = greedy_best_first_search(puzzle, manhattan_distance)

    # Call Greedy Best-First search with misplaced tiles heuristic
    solution_gbfs_misplaced = greedy_best_first_search(puzzle, misplaced_tiles)

    return solution_astar_manhattan, solution_astar_misplaced, solution_gbfs_manhattan, solution_gbfs_misplaced

def scramble():
    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()

def solve_steps():
    global puzzle
    global state
    initial_state = list(puzzle.initial)
    solution = solve(initial_state, goal_state)  # Assume goal_state is defined globally

    def update_ui(moves, current_index, current_state):
        global state
        global puzzle
        if moves[current_index] in puzzle.actions(current_state):
            new_state = list(puzzle.result(current_state, moves[current_index]))
            puzzle = EightPuzzle(tuple(new_state))
            state = new_state
            create_buttons()
            root.update()
            if current_index + 1 < len(moves):
                root.after(750, update_ui, moves, current_index + 1, new_state)

    if solution and len(solution[0]) > 0:
        root.after(750, update_ui, solution[0], 0, initial_state)

def exchange(index):
    global state
    global puzzle
    zero_ix = state.index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))
        create_buttons()

def create_buttons():
    for i in range(9):
        b[i] = Button(root, text=f'{state[i]}' if state[i] != 0 else '', width=6,
                      font=('Helvetica', 40, 'bold'), command=partial(exchange, i))
        b[i].grid(row=i // 3, column=i % 3, ipady=40)

def create_static_buttons():
    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(scramble))
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps))
    solve_btn.grid(row=3, column=2, ipady=10)

# Assuming goal_state is defined globally, you can set it as per your requirement
goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]

def init():
    global state
    global puzzle
    state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    puzzle = EightPuzzle(tuple(state))
    scramble()
    create_buttons()
    create_static_buttons()


In [4]:

# Main application loop
root = Tk()
root.title('Eight Puzzle Solver')
b = [None] * 9
state = []
puzzle = None
init()
root.mainloop()
